
# Reliability Testing Trigger Diagnostic tool

In [1]:
import pandas as pd
from datetime import datetime

C:\Users\athompso\AppData\Local\Temp\ipykernel_38496\1524203933.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



## Testing Reliability at 1HZ, 50% duty

In [6]:
# Read the log data from the file
with open('1hz_50percent_test.txt', 'r') as file:
    log_data = file.read()

# sample data format:
# 2021-01-01 10:03:00::high::36179721
# 2021-01-01 10:03:01::low::36180225

# Split the log data into lines
lines = log_data.split('\n')

In [10]:
# Initialize lists to store the parsed data
timestamps = []
trigger_states = []
times = []

# Initialize a variable to keep track of the previous trigger state
previous_trigger = None

# Initialize a counter for skipped triggers
skipped_triggers = 0

# Iterate over the lines and parse the data
for idx, line in enumerate(lines):
    if line.strip():  # Check if the line is not empty
        timestamp_str, trigger, value_str = line.split('::')
        timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
        value = int(value_str)
        
        # Check if the current trigger state is the same as the previous one
        if previous_trigger is not None and previous_trigger == trigger:
            print(f"Warning: Consecutive '{trigger}' triggers found at rows {idx} and {idx+1}")
            skipped_triggers += 1
            if trigger == 'high':
                # Remove the previous 'high' trigger from the lists
                timestamps.pop()
                trigger_states.pop()
                times.pop()
            continue
        
        timestamps.append(timestamp)
        trigger_states.append(trigger)
        times.append(value)
        
        previous_trigger = trigger

# Create a DataFrame from the parsed data
df = pd.DataFrame({
    'timestamp': timestamps,
    'trigger state': trigger_states,
    'time (ms)': times
})

# Initialize lists to store the new DataFrame data
high_timestamps = []
low_timestamps = []
trigger_lengths = []
time_since_previous_triggers = []

# Initialize variables to keep track of previous timestamps and values
previous_low_time = None

# Iterate over the DataFrame and create the new DataFrame
for i in range(0, len(df) - 1, 2):
    high_timestamp = df.iloc[i]['timestamp']
    low_timestamp = df.iloc[i+1]['timestamp']
    trigger_length = df.iloc[i+1]['time (ms)'] - df.iloc[i]['time (ms)']
    
    if previous_low_time is not None:
        time_since_previous_trigger = df.iloc[i]['time (ms)'] - previous_low_time
    else:
        time_since_previous_trigger = None
    
    high_timestamps.append(high_timestamp)
    low_timestamps.append(low_timestamp)
    trigger_lengths.append(trigger_length)
    time_since_previous_triggers.append(time_since_previous_trigger)
    
    previous_low_time = df.iloc[i+1]['time (ms)']

# Create the new DataFrame
new_df = pd.DataFrame({
    'trigger high timestamp': high_timestamps,
    'trigger low timestamp': low_timestamps,
    'trigger length': trigger_lengths,
    'time since previous trigger': time_since_previous_triggers
})

# print(new_df)
print(f"Total number of triggers skipped: {skipped_triggers}")


Total number of triggers skipped: 20


In [11]:
print(new_df)

      trigger high timestamp trigger low timestamp  trigger length  \
0        2021-01-01 00:00:05   2021-01-01 00:00:06             503   
1        2021-01-01 00:00:06   2021-01-01 00:00:07             503   
2        2021-01-01 00:00:07   2021-01-01 00:00:08             503   
3        2021-01-01 00:00:08   2021-01-01 00:00:09             503   
4        2021-01-01 00:00:09   2021-01-01 00:00:10             503   
...                      ...                   ...             ...   
35883    2021-01-01 10:05:03   2021-01-01 10:05:03             503   
35884    2021-01-01 10:05:04   2021-01-01 10:05:04             503   
35885    2021-01-01 10:05:05   2021-01-01 10:05:05             503   
35886    2021-01-01 10:05:06   2021-01-01 10:05:06             503   
35887    2021-01-01 10:05:07   2021-01-01 10:05:07             503   

       time since previous trigger  
0                              NaN  
1                            503.0  
2                            504.0  
3          

In [13]:
new_df.describe()

,trigger high timestamp,trigger low timestamp,trigger length,time since previous trigger
count,35888,35888,35888.000000,35887.000000
mean,2021-01-01 05:02:30.346772992,2021-01-01 05:02:30.851454464,506.183181,505.367710
min,2021-01-01 00:00:05,2021-01-01 00:00:06,100.000000,154.000000
25%,2021-01-01 02:31:17.750000128,2021-01-01 02:31:18.750000128,503.000000,503.000000
50%,2021-01-01 05:02:25.500000,2021-01-01 05:02:26.500000,503.000000,503.000000
75%,2021-01-01 07:33:35.249999872,2021-01-01 07:33:35.249999872,503.000000,504.000000
max,2021-01-01 10:05:07,2021-01-01 10:05:07,8053.000000,8052.000000
std,NaN,NaN,115.393391,101.868186


In [15]:
print(len(new_df[new_df['trigger length'] > 600]))

30


**Observations**
- a number of trigger events were missed in the logs, shown by the 30 lines with trigger lengths above 600, most with times in the 1000's. 
- changes: 
    - increase pico clock speed
    - switch to mqtt data transfer
    - if needed after mqtt, implement redundant logging and data transfer check with retry if fail
    